# **Machine Learning | Algoritmos Genéticos**

Aplicam uma abordagem evolutiva (seleção natural das espécies) para a aprendizagem de máquina indutiva. Ou seja, ele aplica princípios evolutivos, como seleção, recombinação e mutação, em uma população de soluções candidatas.
Então, basicamente, o algoritmo vai tentar encontrar várias soluções e usar a informação obtida (função objetivo) para conseguir soluções cada vez melhores.

Para isso, vamos usar a biblioteca deap, cuja documentação pode ser vista no link a seguir: https://deap.readthedocs.io/en/master/overview.html

# Problema do caixeiro-viajante

O TSP, ou problema do caixeiro-viajante em português, representado na figura, consiste na procura de um circuito que possua a menor distância, começando numa cidade (ou local) qualquer, entre várias, visitando cada cidade (local) precisamente uma vez e regressando à cidade (local) inicial.

# Aplicação

In [1]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.9 MB/s eta 0:00:00


In [2]:
# Imports
import random
import numpy as np
from deap import algorithms, base, creator, tools
import folium

In [3]:
# Criando um array com as coordenadas das 20 principais cidades européias
cidades = np.array([
    [51.5074, -0.1278],     # Londres
    [48.8566, 2.3522],      # Paris
    [41.9028, 12.4964],     # Roma
    [52.5200, 13.4050],     # Berlim
    [55.7558, 37.6176],     # Moscou
    [40.4168, -3.7038],     # Madri
    [52.3702, 4.8952],      # Amsterdã
    [59.3293, 18.0686],     # Estocolmo
    [38.7223, -9.1393],     # Lisboa
    [48.2082, 16.3738],     # Viena
    [53.3498, -6.2603],     # Dublin
    [41.0082, 28.9784],     # Istambul
    [52.2297, 21.0122],     # Varsóvia
    [47.4979, 19.0402],     # Budapeste
    [59.9343, 30.3351],     # São Petersburgo
    [59.9139, 10.7522],     # Oslo
    [50.0755, 14.4378],     # Praga
    [45.4642, 9.1900],      # Milão
    [48.2082, 11.6680],     # Munique
    [55.6761, 12.5683],     # Copenhague
])

In [ ]:
# Cálculo do centro
centro = np.mean(cidades, axis=0)

# Criar um novo objeto folium.Map com base no centro
m = folium.Map(location=centro, zoom_start=4)

# Adicionar marcadores para cada cidade
for cidade in cidades:
    folium.Marker([cidade[0], cidade[1]]).add_to(m)

m

Saída: https://github.com/mariaraquelbarbosa/genetic-algorithm/blob/main/mapa_interativo_v1.html

In [5]:
m.save('mapa_interativo_v1.html')

In [6]:
# Cálculo da distância entre duas cidades
def distancia(cidade1, cidade2):
    return np.linalg.norm(cidade1 - cidade2)

# Cálculo da distância total da rota
def distancia_total_rota(rota):
    distancia_total = 0
    for i in range(len(rota) - 1):
        cidade1 = cidades[rota[i]]
        cidade2 = cidades[rota[i + 1]]
        distancia_total += distancia(cidade1, cidade2)
    return distancia_total

# Função de avaliação ou de otimização (fitness)
def funcao_avaliacao(individuo):
    rota = individuo + [individuo[0]]  # Adiciona a cidade inicial no final
    return (distancia_total_rota(rota),)

In [7]:
# Definição do tipo de problema
creator.create("MyFitnessMin", base.Fitness, weights=(-1.0,)) # o objetivo é encontrar a solução com o menor valor de aptidão possível (problema de minimização)
creator.create("MyIndividual", list, fitness=creator.MyFitnessMin) # representa um indivíduo da população, ou seja, uma solução candidata ao problema de otimização

# Criando inicializadores e operadores das ferramentas do DEAP
toolbox = base.Toolbox() # Instância da classe Toolbox que contém as funções de operadores genéticos (como seleção, reprodução, mutação) e as funções de avaliação necessárias para o algoritmo
toolbox.register("indices", random.sample, range(len(cidades)), len(cidades)) # Gera uma lista de índices únicos aleatoriamente em um intervalo de 0 até 20 (len(cidades))
toolbox.register("individual", tools.initIterate, creator.MyIndividual, toolbox.indices) # Cria os indivíduos do algoritmo genético
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # Cria uma população/uma lista de indivíduos
toolbox.register("evaluate", funcao_avaliacao) # Calcula o valor de aptidão (fitness) de um indivíduo
toolbox.register("mate", tools.cxOrdered) # Rreprodução entre dois indivíduos (está sendo utilizado o operador cxOrdered para realizar o cruzamento ordenado)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # Mutação em um indivíduo (está sendo utilizado mutShuffleIndexes para embaralha os índices com uma probabilidade de mutação de 0.05)
toolbox.register("select", tools.selTournament, tournsize=3) # Seleção de indivíduos para reprodução (está sendo utilizado selTournament para selecionar os indivíduos por um torneio com tamanho igual a 3)

# População inicial de tamanho 100
populacao = toolbox.population(n=100)

# Execução do algoritmo genético
algorithms.eaSimple(populacao, toolbox, cxpb=0.7, mutpb=0.2, ngen=500, verbose=False)
  # cxpb é a probabilidade de ocorrência do crossover (reprodução) entre dois indivíduos
  # mutpb é a probabilidade de ocorrência da mutação em um indivíduo
  # ngen é o número de iterações do algoritmo genético
  # verbose é um parâmetro opcional que controla a exibição de informações durante a execução do algoritmo

# Obtenção do melhor indivíduo da população final
melhor_individuo = tools.selBest(populacao, k=1)[0] # k=1 indica que será selecionado apenas o melhor indivíduo
melhor_individuo = melhor_individuo + [melhor_individuo[0]]  # Adiciona a cidade inicial no final

# Visualização do resultado
print("Melhor rota encontrada:", melhor_individuo)
print("Distância da melhor rota:", funcao_avaliacao(melhor_individuo))

Melhor rota encontrada: [18, 17, 6, 1, 0, 10, 8, 5, 2, 16, 9, 13, 12, 11, 4, 14, 7, 15, 19, 3, 18]
Distância da melhor rota: (153.44066982950363,)


In [8]:
# Lista de nomes das cidades
cidades_nomes = ["Londres",
                 "Paris",
                 "Roma",
                 "Berlim",
                 "Moscou",
                 "Madri",
                 "Amsterdã",
                 "Estocolmo",
                 "Lisboa",
                 "Viena",
                 "Dublin",
                 "Istambul",
                 "Varsóvia",
                 "Budapeste",
                 "São Petersburgo",
                 "Oslo",
                 "Praga",
                 "Milão",
                 "Munique",
                 "Copenhague"]

# List comprehension para colocar nome das cidade na rota final
melhor_rota_nomes = [cidades_nomes[cidade] for cidade in melhor_individuo]

# Print no resultado com os nomes das cidades
print("Melhor rota encontrada:", melhor_rota_nomes)
print("Distância da melhor rota:", distancia_total_rota(melhor_individuo))

Melhor rota encontrada: ['Munique', 'Milão', 'Amsterdã', 'Paris', 'Londres', 'Dublin', 'Lisboa', 'Madri', 'Roma', 'Praga', 'Viena', 'Budapeste', 'Varsóvia', 'Istambul', 'Moscou', 'São Petersburgo', 'Estocolmo', 'Oslo', 'Copenhague', 'Berlim', 'Munique']
Distância da melhor rota: 153.44066982950363


In [9]:
# Cria um novo objeto `folium.Map` com base no centro
m = folium.Map(location=centro, zoom_start=4)

# Cria uma lista `rota_final_coordenadas`
rota_final_coordenadas = [[cidades[cidade][0], cidades[cidade][1]] for cidade in melhor_individuo]
rota_final_coordenadas

[[48.2082, 11.668],
 [45.4642, 9.19],
 [52.3702, 4.8952],
 [48.8566, 2.3522],
 [51.5074, -0.1278],
 [53.3498, -6.2603],
 [38.7223, -9.1393],
 [40.4168, -3.7038],
 [41.9028, 12.4964],
 [50.0755, 14.4378],
 [48.2082, 16.3738],
 [47.4979, 19.0402],
 [52.2297, 21.0122],
 [41.0082, 28.9784],
 [55.7558, 37.6176],
 [59.9343, 30.3351],
 [59.3293, 18.0686],
 [59.9139, 10.7522],
 [55.6761, 12.5683],
 [52.52, 13.405],
 [48.2082, 11.668]]

In [ ]:
# Cria uma linha no mapa `m` utilizando as coordenadas da rota
folium.PolyLine(rota_final_coordenadas, color='blue', weight=2, opacity=1).add_to(m)
m

Saída: https://github.com/mariaraquelbarbosa/genetic-algorithm/blob/main/mapa_interativo_v2.html

In [11]:
m.save('mapa_interativo_v2.html')

# **Considerações finais**

O algoritmo genético tem uma aplicabilidade mais específica, quando comparado com outros algoritmos de machine learning e, no caso do TSP, não considerou muitos fatores que podem determinantes ao tornar este exemplo um real problema de logística, como trabalhar com rotas de transporte reais, ou ainda considerar o número de pedidos por cidade e a capacidade de cada veículo de transporte. Ainda assim, ele pode gerar um ponto de partida para desenvolver uma solução ótima nesse tipo de problema.

Podemos aplicá-lo em outros contextos, como:
* Seleção de carteiras de investimento (maximizar o retorno e minimizar o risco).
* Problemas de programação combinatória, como escalonamento de horários, alocação de recursos, agendamento de tarefas.
* Otimização de projetos de circuitos eletrônicos.